First, install the fhir.resources library.

In [1]:
!pip install fhir.resources

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.7/307.7 kB 8.1 MB/s eta 0:00:00


The following script will extract observation resources from a questionnaire response resource.

In [22]:
import json
from fhir.resources.observation import Observation
from fhir.resources.codeableconcept import CodeableConcept
from fhir.resources.coding import Coding

def extract_units(questionnaire_file):
    units_map = {}
    with open(questionnaire_file, 'r') as f:
        questionnaire = json.load(f)
        for item in questionnaire['item']:
            link_id = item['linkId']
            if 'extension' in item:
                for extension in item['extension']:
                    if 'url' in extension and extension['url'] == 'http://hl7.org/fhir/StructureDefinition/questionnaire-unit':
                        units_map[link_id] = extension['valueCoding']['display']
    return units_map

def create_observation(link_id, text, answer, units=None):
    # Set code representing the question
    code = CodeableConcept(
        coding=[Coding(
            system="http://loinc.org",
            code=link_id.lstrip('/'),
            display=text
        )]
    )

    # Set status to 'final' as these are completed observations
    observation = Observation(code=code, status="final")

    # Set value based on the answer type
    if 'valueDecimal' in answer:
        value = float(answer['valueDecimal'])
        if units:  # Check if units are provided
            observation.valueQuantity = {
                "value": value,
                "unit": units
            }
        else:
            observation.valueQuantity = {"value": value}
    elif 'valueCodeableConcept' in answer:
        value_codeable_concept = CodeableConcept(
            coding=[Coding(
                system=answer['valueCodeableConcept']['system'],
                code=answer['valueCodeableConcept']['code'],
                display=answer['valueCodeableConcept']['display']
            )]
        )
        observation.valueCodeableConcept = value_codeable_concept

    return observation

def main():
    # Extract units from questionnaire file
    units_map = extract_units('ht_wt_panel-q.json')

    # Parse questionnaire response JSON file
    with open('ht_wt_panel-qr.json', 'r') as f:
        questionnaire_response = json.load(f)

    # Iterate through each item in the questionnaire response
    for item in questionnaire_response['item']:
        link_id = item['linkId']
        text = item['text']
        answer = item['answer'][0]

        # Get units for the current question
        units = units_map.get(link_id)

        # Create Observation resource for the item
        observation = create_observation(link_id, text, answer, units)

        # Print the observation
        print(observation.json(indent=4))

if __name__ == "__main__":
    main()


{
    "resourceType": "Observation",
    "status": "final",
    "code": {
        "coding": [
            {
                "system": "http://loinc.org",
                "code": "9732277310580",
                "display": "What brings you to the doctor today?"
            }
        ]
    }
}
{
    "resourceType": "Observation",
    "status": "final",
    "code": {
        "coding": [
            {
                "system": "http://loinc.org",
                "code": "29463-7",
                "display": "What is your weight?"
            }
        ]
    },
    "valueQuantity": {
        "value": 90.0,
        "unit": "kg"
    }
}
{
    "resourceType": "Observation",
    "status": "final",
    "code": {
        "coding": [
            {
                "system": "http://loinc.org",
                "code": "8302-2",
                "display": "What is your height?"
            }
        ]
    },
    "valueQuantity": {
        "value": 72.0,
        "unit": "[in_i]"
    }
}
